## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import tabula

## Read-in

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.io.read_pdf('condo.pdf', pages='all',lattice=True)

## Clean Data

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## Geocode

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [6]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [7]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction section

In [8]:
df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,geocoded,lat,lon
1,Elysee\r788 NE 23rd St 5601\rMiami,6/5/2023,160,"$5,500,000.00",3490,"$1,575.93",Brenda Bichili,Douglas Elliman,Jason Zarco,BHHS EWM Realty,Elysee 788 NE 23rd St 5601 Miami,Elysee,788 NE 23rd St 5601 Miami,"(25.7990967, -80.18572309999999)",25.799097,-80.185723
2,Parque Towers\r330 Sunny Isles Blvd 5-TS04\rSu...,6/9/2023,186,"$4,450,000.00",4736,$939.61,Wendy Cohen,One Sotheby's International Realty,Mirta Branco,LPT Realty,Parque Towers 330 Sunny Isles Blvd 5-TS04 Sunn...,Parque Towers,330 Sunny Isles Blvd 5-TS04 Sunny Isles Beach,"(25.9294643, -80.1259521)",25.929464,-80.125952
3,Harborview at Fisher Island\r4422 Fisher Islan...,6/9/2023,8,"$4,250,000.00",2744,"$1,548.83",Lourdes Alatriste,Douglas Elliman,Karla Abaunza,Luxury Living Realty,Harborview at Fisher Island 4422 Fisher Island...,Harborview at Fisher Island,4422 Fisher Island Dr 4422 Miami Beach,"(25.7609678, -80.1449375)",25.760968,-80.144937
4,Jade Signature\r16901 Collins Ave 2303\rSunny ...,6/6/2023,49,"$3,550,000.00",2496,"$1,422.28",Sergio Bu PLLC,Beachfront Realty Inc,Rodney Mendes Irala,Coldwell Banker Realty,Jade Signature 16901 Collins Ave 2303 Sunny Is...,Jade Signature,16901 Collins Ave 2303 Sunny Isles Beach,"(25.9327557, -80.1210054)",25.932756,-80.121005
5,Muse\r17141 Collins Ave 4702\rSunny Isles Beach,6/8/2023,62,"$3,500,000.00",2042,"$1,714.01",Victoria Romanenko,Golden Keys Inc,Sarah Gozlan,"G Realty Group, LLC.",Muse 17141 Collins Ave 4702 Sunny Isles Beach,Muse,17141 Collins Ave 4702 Sunny Isles Beach,"(25.9349392, -80.12091819999999)",25.934939,-80.120918
6,Millennium\r18671 Collins Ave 3102\rSunny Isle...,6/6/2023,42,"$2,500,000.00",2230,"$1,121.08",Patricia Munoz,United Realty Group Inc,Patricia Munoz,United Realty Group Inc,Millennium 18671 Collins Ave 3102 Sunny Isles ...,Millennium,18671 Collins Ave 3102 Sunny Isles Beach,"(25.9486992, -80.1197559)",25.948699,-80.119756
7,Santa Maria\r1643 Brickell Ave 1603\rMiami,6/7/2023,109,"$2,200,000.00",2180,"$1,009.17",Alejandro Cure,"Miami One Real Estate, LLC",Nicolas Pollini PA,BHHS EWM Realty,Santa Maria 1643 Brickell Ave 1603 Miami,Santa Maria,1643 Brickell Ave 1603 Miami,"(25.7554036, -80.195048)",25.755404,-80.195048
8,Peninsula\r3201 NE 183 St 3003\rAventura,6/8/2023,193,"$2,000,000.00",3464,$612.37,Lana Bell,Douglas Elliman,Marcelo Kohen,One Sotheby's International Realty,Peninsula 3201 NE 183 St 3003 Aventura,Peninsula,3201 NE 183 St 3003 Aventura,"(25.9456839, -80.13577169999999)",25.945684,-80.135772
9,Towers of Key Biscayne\r1121 Crandon Blvd D120...,6/7/2023,63,"$1,915,000.00",1782,"$1,074.64",Carmen B Llano White PA,BHHS EWM Realty,Jill Olcott,Keller Williams Miami Beach Realty,Towers of Key Biscayne 1121 Crandon Blvd D1208...,Towers of Key Biscayne,1121 Crandon Blvd D1208 Key Biscayne,"(25.6834508, -80.15741469999999)",25.683451,-80.157415
10,01500 Ocean Drive\r1500 Ocean Dr 1107\rMiami B...,6/7/2023,27,"$1,835,000.00",1170,"$1,568.38",Ines Flax,One Sotheby's International Realty,Meredith Tobin,Thine Agency,01500 Ocean Drive 1500 Ocean Dr 1107 Miami Beach,,01500 Ocean Drive 1500 Ocean Dr 1107 Miami Beach,"(25.787803, -80.1292869)",25.787803,-80.129287


In [9]:
df.at[10,'building_name']=('1500 Ocean Drive')
df.at[10,'address_only']=('1500 Ocean Drive')

## Format Data

In [10]:
### Insert NaNs if needed ###
df = df.replace('N/A', np.nan)

In [11]:
df['Sale Price']

1     $5,500,000.00
2     $4,450,000.00
3     $4,250,000.00
4     $3,550,000.00
5     $3,500,000.00
6     $2,500,000.00
7     $2,200,000.00
8     $2,000,000.00
9     $1,915,000.00
10    $1,835,000.00
Name: Sale Price, dtype: object

In [12]:
df['int_Sale_Price'] = df['Sale Price'].str.replace('$','',regex=False)

In [13]:
df['int_Sale_Price'] = df['int_Sale_Price'].str.replace(',','',regex=False)

In [14]:
df['int_Sale_Price'] = pd.to_numeric(df['int_Sale_Price'])

In [15]:
df['price_per_sqft'] = df['price_per_sqft'].str.replace('$','',regex=False)
df['price_per_sqft'] = df['price_per_sqft'].str.replace(',','',regex=False)
df['price_per_sqft'] = pd.to_numeric(df['price_per_sqft'])

df['days_on_market'] = pd.to_numeric(df['days_on_market'])

## Color-code top sale

In [16]:
### Insert RANK values ###
df['RANK'] = range(1, len(df) + 1)
# use numpy to assign values to the 'COLOR' column
df['COLOR'] = np.where(df['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [17]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only',
       'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR'],
      dtype='object')

In [18]:
def popup_html(row):
    building_name = row['building_name']
    price = row['Sale Price']
    days_on_market = row['days_on_market']
    listing_agent = row['Agent']
    buyers_agent = row['Buyer Agent']
    psf = row['price_per_sqft']
    address = row['address_only']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>{}'''.format(price) + '''<br>
    <strong>Days on Market: </strong>{}'''.format(days_on_market) + '''<br>
    <strong>Listing Agent: </strong>{}'''.format(listing_agent) + '''<br>
    <strong>Buyer's Agent: </strong>{}'''.format(buyers_agent) + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(psf) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    </html>
    '''
    return html

In [19]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('June 4th - June 10th')


### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [20]:
df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,geocoded,lat,lon,int_Sale_Price,RANK,COLOR
1,Elysee\r788 NE 23rd St 5601\rMiami,6/5/2023,160,"$5,500,000.00",3490,1575.93,Brenda Bichili,Douglas Elliman,Jason Zarco,BHHS EWM Realty,Elysee 788 NE 23rd St 5601 Miami,Elysee,788 NE 23rd St 5601 Miami,"(25.7990967, -80.18572309999999)",25.799097,-80.185723,5500000.0,1,orange
2,Parque Towers\r330 Sunny Isles Blvd 5-TS04\rSu...,6/9/2023,186,"$4,450,000.00",4736,939.61,Wendy Cohen,One Sotheby's International Realty,Mirta Branco,LPT Realty,Parque Towers 330 Sunny Isles Blvd 5-TS04 Sunn...,Parque Towers,330 Sunny Isles Blvd 5-TS04 Sunny Isles Beach,"(25.9294643, -80.1259521)",25.929464,-80.125952,4450000.0,2,blue
3,Harborview at Fisher Island\r4422 Fisher Islan...,6/9/2023,8,"$4,250,000.00",2744,1548.83,Lourdes Alatriste,Douglas Elliman,Karla Abaunza,Luxury Living Realty,Harborview at Fisher Island 4422 Fisher Island...,Harborview at Fisher Island,4422 Fisher Island Dr 4422 Miami Beach,"(25.7609678, -80.1449375)",25.760968,-80.144937,4250000.0,3,blue
4,Jade Signature\r16901 Collins Ave 2303\rSunny ...,6/6/2023,49,"$3,550,000.00",2496,1422.28,Sergio Bu PLLC,Beachfront Realty Inc,Rodney Mendes Irala,Coldwell Banker Realty,Jade Signature 16901 Collins Ave 2303 Sunny Is...,Jade Signature,16901 Collins Ave 2303 Sunny Isles Beach,"(25.9327557, -80.1210054)",25.932756,-80.121005,3550000.0,4,blue
5,Muse\r17141 Collins Ave 4702\rSunny Isles Beach,6/8/2023,62,"$3,500,000.00",2042,1714.01,Victoria Romanenko,Golden Keys Inc,Sarah Gozlan,"G Realty Group, LLC.",Muse 17141 Collins Ave 4702 Sunny Isles Beach,Muse,17141 Collins Ave 4702 Sunny Isles Beach,"(25.9349392, -80.12091819999999)",25.934939,-80.120918,3500000.0,5,blue
6,Millennium\r18671 Collins Ave 3102\rSunny Isle...,6/6/2023,42,"$2,500,000.00",2230,1121.08,Patricia Munoz,United Realty Group Inc,Patricia Munoz,United Realty Group Inc,Millennium 18671 Collins Ave 3102 Sunny Isles ...,Millennium,18671 Collins Ave 3102 Sunny Isles Beach,"(25.9486992, -80.1197559)",25.948699,-80.119756,2500000.0,6,blue
7,Santa Maria\r1643 Brickell Ave 1603\rMiami,6/7/2023,109,"$2,200,000.00",2180,1009.17,Alejandro Cure,"Miami One Real Estate, LLC",Nicolas Pollini PA,BHHS EWM Realty,Santa Maria 1643 Brickell Ave 1603 Miami,Santa Maria,1643 Brickell Ave 1603 Miami,"(25.7554036, -80.195048)",25.755404,-80.195048,2200000.0,7,blue
8,Peninsula\r3201 NE 183 St 3003\rAventura,6/8/2023,193,"$2,000,000.00",3464,612.37,Lana Bell,Douglas Elliman,Marcelo Kohen,One Sotheby's International Realty,Peninsula 3201 NE 183 St 3003 Aventura,Peninsula,3201 NE 183 St 3003 Aventura,"(25.9456839, -80.13577169999999)",25.945684,-80.135772,2000000.0,8,blue
9,Towers of Key Biscayne\r1121 Crandon Blvd D120...,6/7/2023,63,"$1,915,000.00",1782,1074.64,Carmen B Llano White PA,BHHS EWM Realty,Jill Olcott,Keller Williams Miami Beach Realty,Towers of Key Biscayne 1121 Crandon Blvd D1208...,Towers of Key Biscayne,1121 Crandon Blvd D1208 Key Biscayne,"(25.6834508, -80.15741469999999)",25.683451,-80.157415,1915000.0,9,blue
10,01500 Ocean Drive\r1500 Ocean Dr 1107\rMiami B...,6/7/2023,27,"$1,835,000.00",1170,1568.38,Ines Flax,One Sotheby's International Realty,Meredith Tobin,Thine Agency,01500 Ocean Drive 1500 Ocean Dr 1107 Miami Beach,1500 Ocean Drive,1500 Ocean Drive,"(25.787803, -80.1292869)",25.787803,-80.129287,1835000.0,10,blue


In [21]:
m.save('index.html')

## Data snagger

In [22]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [25]:
### Highest and lowest sale price ###
print(f"{ME}{BR}{df.loc[df['int_Sale_Price'].idxmax()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmax()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmax()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmax()]['days_on_market']}")
print(f"{LE}{BR}{df.loc[df['int_Sale_Price'].idxmin()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmin()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmin()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmin()]['days_on_market']}")
### Highest and lowest psf ###
print(f"{MAX_PSF}{BR}{df.loc[df['price_per_sqft'].idxmax()]['building_name']}, {df.loc[df['price_per_sqft'].idxmax()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmax()]['Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmax()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmax()]['days_on_market']}")
print(f"{MIN_PSF}{BR}{df.loc[df['price_per_sqft'].idxmin()]['building_name']}, {df.loc[df['price_per_sqft'].idxmin()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmin()]['Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmin()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmin()]['days_on_market']}")
### Highest and lowest days on market ###
print(f"{DAYS_MAX}{BR}{df.loc[df['days_on_market'].idxmax()]['building_name']}, {df.loc[df['days_on_market'].idxmax()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmax()]['Agent']} with {df.loc[df['days_on_market'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmax()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmax()]['days_on_market']}")
print(f"{DAYS_MIN}{BR}{df.loc[df['days_on_market'].idxmin()]['building_name']}, {df.loc[df['days_on_market'].idxmin()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmin()]['Agent']} with {df.loc[df['days_on_market'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmin()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmin()]['days_on_market']}")

Most Expensive
Ritz Carlton , 4701 N Meridian Ave 601 Miami Beach | Price $7,400,000 | $1,783 psf | Listing agent: Dina Goldentayer with Douglas Elliman | Buyer's agent: Darin Tansey with Douglas Elliman | Days on market: 233
Least Expensive
Turnberry Ocean Colony , 16051 Collins Ave 3301 Sunny Isles Beach | Price $2,900,000 | $1,045 psf | Listing agent: Bento Queiroz with Compass Florida, LLC. | Buyer's agent: Hector Stracuzzi with Beachfront Realty Inc | Days on market: 106
Highest Price Per Square Foot
57 Ocean, 5775 Collins Ave | Price $5,550,000 | $3,212 psf | Listing agent: Christopher Jude with Fortune Christie's Int'l Real Estate | Buyer's agent: Christopher Jude with Fortune Christie's Int'l Real Estate | Days on market: 41
Lowest Price Per Square Foot
Grand Bay, 445 Grand Bay | Price $2,900,000 | $983 psf | Listing agent: Douglas Kinsley with Fortune Christie's Int'l Real Estate | Buyer's agent: Maria Jarque with Fortune Christie's Int'l Real Estate | Days on market: 25
Most 

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [23]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [24]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_06052023
